# Poisoning Attack in Federated Learning Experiment 1.1

Purpose: 
- What is the performance loss induced through the different poisoning attack? 
- What is the performance of the global model without poisoning?


## Google Colab

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
sys.path.append('/content/drive/My Drive/Colab Notebooks/federated_learning')

In [ ]:
!pip install shap

## Libraries

In [ ]:
from federated_learning.utils import SHAPUtil, experiment_util
from federated_learning import ClientPlane, Configuration, ObserverConfiguration
from federated_learning.server import Server

## Additional Functions

In [ ]:
class Experiment():
    def __init__(self, num_p_clients, from_label, to_label, natural_label ):
        self.num_p_clients = num_p_clients
        self.from_label = from_label
        self.to_label = to_label
        self.neutral_label = natural_label
        self.accuracy = []
        self.recall = {}
        self.initialize_recall()

    def initialize_recall(self):
        self.recall[self.from_label] = []
        self.recall[self.to_label] = []
        self.recall[self.neutral_label] = []
        

In [ ]:
def add_round_to_experiment(experiment, recall, accuracy, from_label, to_label, neutral_label): 
    experiment.accuracy.append(accuracy)
    experiment.recall[from_label].append(recall[from_label])
    experiment.recall[to_label].append(recall[to_label])
    experiment.recall[neutral_label].append(recall[neutral_label])

In [ ]:
experiments = []

## MNIST(5,4)

In [ ]:
from federated_learning.nets import MNISTCNN
from federated_learning.dataset import MNISTDataset
import os
config = Configuration()
config.FROM_LABEL = 5
config.TO_LABEL = 4
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = MNISTDataset
config.MODELNAME = config.MNIST_NAME
config.NETWORK = MNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.datasetObserverConfiguration = "MNIST"
neutral_label = 2

In [ ]:
# Google Colab config
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"

In [ ]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)

In [ ]:
import numpy as np
import copy
for num_p_clients in [0, 10, 25, 50, 75, 100, 200]:
    client_plane.reset_default_client_nets()
    server.reset_to_default_net()
    config.POISONED_CLIENTS = num_p_clients
    experiment = Experiment(num_p_clients, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    server.test()
    recall, precision, accuracy = server.analize_test()
    add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    for i in range(200):
        experiment_util.set_rounds(client_plane, server, i+1)
        experiment_util.run_round(client_plane, server, i+1)
        if (i+1)%10 == 0:
            server.test()
            recall, precision, accuracy = server.analize_test()
            add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
            print("Round {} finished".format(i+1))
    print(experiment.accuracy)
    print(experiment.recall[config.FROM_LABEL])
    print(experiment.recall[config.TO_LABEL])
    print(experiment.recall[neutral_label])
    experiments.append(copy.deepcopy(experiment))
    client_plane.reset_poisoning_attack()

In [ ]:
for e in experiments: 
    print(e.accuracy)
    print(e.recall[config.FROM_LABEL])
    print(e.recall[config.TO_LABEL])
    print(e.recall[neutral_label])

# Fashion MNIST(5,4)

In [ ]:
from federated_learning.nets import FMNISTCNN
from federated_learning.dataset import FMNISTDataset
import os
config = Configuration()
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"
config.FROM_LABEL = 5
config.TO_LABEL = 4
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = FMNISTDataset
config.MODELNAME = config.FMNIST_NAME
config.NETWORK = FMNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.dataset = "FMNIST"
neutral_label = 2

In [ ]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)
experiments_fmnist = []

In [ ]:
import numpy as np
import copy
for num_p_clients in [0, 10, 25, 50, 75, 100, 200]:
    client_plane.reset_default_client_nets()
    server.reset_to_default_net()
    client_plane.reset_poisoning_attack()
    config.POISONED_CLIENTS = num_p_clients
    experiment = Experiment(num_p_clients, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    server.test()
    recall, precision, accuracy = server.analize_test()
    add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    for i in range(200):
        experiment_util.set_rounds(client_plane, server, i+1)
        experiment_util.run_round(client_plane, server, i+1)
        if (i+1)%10 == 0:
            server.test()
            recall, precision, accuracy = server.analize_test()
            add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
            print("Round {} finished".format(i+1))
    print(experiment.accuracy)
    print(experiment.recall[config.FROM_LABEL])
    print(experiment.recall[config.TO_LABEL])
    print(experiment.recall[neutral_label])
    experiments_fmnist.append(copy.deepcopy(experiment))

In [ ]:
for e in experiments_fmnist: 
    print(e.accuracy)
    print(e.recall[config.FROM_LABEL])
    print(e.recall[config.TO_LABEL])
    print(e.recall[neutral_label])